In [4]:
!pip3 install --user underthesea
import regex as re
from underthesea import word_tokenize
import pandas as pd

In [52]:
# read txt file
f = open("input/product_train_0.txt", "r")

# get category and product
categories = []
products = []

while True:
    line = f.readline()
    if not line:
        break
    category = line.split(" ", 1)[0].replace("__label__", "").replace("_", " ")
    categories.append(category)

    product = ''.join(line.split(" ", 1)[1:]).replace("\n", "")
    products.append(product)

# remove duplicate
unique_categories = list(dict.fromkeys(categories))

print(len(categories))
print(len(products))

print(categories[0:5])
print(products[0:5])

14976
14976
['Ghế văn phòng', 'Thước dây', 'Cưa tay', 'Váy, đầm', 'Dụng cụ đo, kiểm tra khác']
['Ghế HQ - HK095', '30m Thước dây làm bằng sợi thủy tinh TOTAL TMTF12306', 'Cưa gỗ cầm tay cán lớn Asaki AK - 8657', 'Đầm ren cổ V tay ngắn cao cấp', 'Máy đo độ đồng tâm HANN YAN 6401D1']


In [53]:
data = pd.DataFrame(list(zip(products, categories)),
                    columns=['products', 'categories'])
print(data.head(5))

                                            products  \
0                                     Ghế HQ - HK095   
1  30m Thước dây làm bằng sợi thủy tinh TOTAL TMT...   
2             Cưa gỗ cầm tay cán lớn Asaki AK - 8657   
3                      Đầm ren cổ V tay ngắn cao cấp   
4                 Máy đo độ đồng tâm HANN YAN 6401D1   

                  categories  
0              Ghế văn phòng  
1                  Thước dây  
2                    Cưa tay  
3                   Váy, đầm  
4  Dụng cụ đo, kiểm tra khác  


# EDA


In [54]:
# check misssing values

data.isna().sum()

products      0
categories    0
dtype: int64

In [55]:
# check data shape

data.shape

(14976, 2)

In [56]:
# check target balance

print(data['categories'].value_counts())

categories
Bao đựng, ốp lưng điện thoại    342
Bàn ghế phòng khách             187
Bộ bàn ghế cafe                 175
Váy, đầm                        158
Bộ bàn ghế ( phòng ăn, bếp )    135
                               ... 
Tời kéo                           1
Van công nghiệp khác              1
Máy in khổ lớn                    1
Bể cá                             1
Bánh pía                          1
Name: count, Length: 1971, dtype: int64


# Data Preprocessing


## 1. Chuẩn hoá unicode


In [57]:
import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()

In [58]:
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

## 2. Chuẩn hoá kiểu gõ dấu

VD: **òa** và **oà** là 2 từ khác nhau. <br>
Chi tiết: [Wikipedia](https://vi.wikipedia.org/wiki/Quy_t%E1%BA%AFc_%C4%91%E1%BA%B7t_d%E1%BA%A5u_thanh_c%E1%BB%A7a_ch%E1%BB%AF_Qu%E1%BB%91c_ng%E1%BB%AF)


In [59]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^p{P}*)([p{L}.]*p{L}+)(p{P}*$)',
                    r'1/2/3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_tắc_đặt_dấu_thanh_trong_chữ_quốc_ngữ
"""
# if __name__ == '__main__':
print(chuan_hoa_dau_cau_tieng_viet('anh hòa hoà thụy thuỵ, đang làm.. gì'))

anh hòa hoà thụy thuỵ, đang làm.. gì


## 3. Tách từ

Học sinh học sinh học => Học_sinh học sinh_học


In [60]:
from underthesea import word_tokenize

sentence = "Tôi là sinh viên trường đại học bách khoa hà nội"
print(word_tokenize(sentence, format="text"))

Tôi là sinh_viên trường đại_học bách_khoa hà_nội


## 4. Đưa về chữ thường


## 5. Xoá kí tự ko cần thiết


# Hàm tiền xử lý


In [61]:
import regex as re


def text_preprocess(document):
    # chuẩn hóa unicode
    # document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    # document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    # document = re.sub(
    #     r'[^swáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', ' ', document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document


documents = 'TP HCM là thành phố lớn nhất Việt Nam'
print(text_preprocess(documents))

tp hcm là thành_phố lớn nhất việt_nam


# Xây dựng mô hình phân loại


## 1. Xây dựng tập train / test


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

test_percent = 0.2

# split data
X_train, X_test, y_train, y_test = train_test_split(data['products'], data['categories'], test_size=test_percent,
                                                    random_state=42)

# save train/test data
with open('data/train.txt', 'w') as f:
    for x, y in zip(X_train, y_train):
        y = y.replace(" ", "_")
        f.write('__label__' + y + ' ' + x + '\n')

with open('data/test.txt', 'w') as f:
    for x, y in zip(X_test, y_test):
        y = y.replace(" ", "_")
        f.write('__label__' + y + ' ' + x + '\n')

# encode target
label_encoder = LabelEncoder()
label_encoder.fit(data['categories'])

print(list(label_encoder.classes_), 'n')

y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

print(X_train)

['Access point (Wifi)', 'Ampe kìm', 'Amplifier', 'Amplifier Hi-End', 'An toàn cho bé khi đi ô tô', 'An toàn cho trẻ', 'Ba lô học sinh', 'Ba lô leo núi, du lịch', 'Baga xe máy', 'Balo thời trang', 'Bao bì thực phẩm', 'Bao cao su', 'Bao cố định chấn thương', 'Bao đựng, ốp lưng điện thoại', 'Barie tự động', 'Bia', 'Biến tần', 'Biến áp máy cấp nguồn đa năng', 'Biển báo giao thông', 'Biển chỉ', 'Biển, bảng, đèn thông báo ( Led )', 'Block điều hòa', 'Bluetooth USB Adapter', 'Board mạch điều khiển', 'Body liền cho bé', 'Boot nữ', 'Bu lông, tắc kê', 'Buồng tắm đa năng', 'Buộc tóc', 'Buộc tóc trẻ em', 'Bài vị thờ', 'Bàn Billiard (Bi A)', 'Bàn cafe ( café)', 'Bàn chơi game - Bàn phòng Game', 'Bàn chải đánh răng', 'Bàn chế biến thực phẩm', 'Bàn chụp sản phẩm', 'Bàn ghế học tập và giảng dạy', 'Bàn ghế nhà hàng, khách sạn', 'Bàn ghế nhựa', 'Bàn ghế phòng khách', 'Bàn ghế phòng ngủ', 'Bàn ghế phòng thí nghiệm', 'Bàn ghế sân vườn', 'Bàn ghế trang điểm', 'Bàn ghế trẻ em', 'Bàn giám đốc', 'Bàn họp', 'B

In [48]:
import os
MODEL_PATH = "models"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

## 2. Naive Bayes

[https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html]


In [49]:
import pickle
import time
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1),
                                              max_df=0.8,
                                              max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "naive_bayes.pkl"), 'wb'))

Done training Naive Bayes in 1.0791888236999512 seconds.


## 3. Linear Classifier

[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html]


In [51]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1),
                                              max_df=0.8,
                                              max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs',
                                                multi_class='auto',
                                                max_iter=10000))
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(
    MODEL_PATH, "linear_classifier.pkl"), 'wb'))

KeyboardInterrupt: 